In [8]:
# %pip install tensorflow==2.12.0

import sys
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MultiLabelBinarizer
from sentence_transformers import SentenceTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from termcolor import colored
import joblib

#print(f"Python version: {sys.version}")
#print(f"Tensorflow version: {tf.__version__}")

In [9]:
# 1. Cargar y preprocesar los datos
df = pd.read_csv("../data/raw/goodreads_data_sample.csv")

# Normalizar columnas necesarias
df['Book'] = df['Book'].fillna('')
df['Description'] = df['Description'].fillna('')
df['Genres'] = df['Genres'].fillna("[]")

# Crear la columna 'text' combinando título y descripción
df['text'] = df['Book'] + ". " + df['Description']

# Convertir la columna 'Genres' de cadenas a listas reales
df['tags'] = df['Genres'].apply(eval)

In [10]:
# 2. Codificar las etiquetas (tags)
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['tags'])

# 3. Convertir texto a embeddings
model_embed = SentenceTransformer('all-MiniLM-L6-v2')
X_embeddings = model_embed.encode(df['text'].tolist(), show_progress_bar=True)

# 4. Dividir los datos en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X_embeddings, y, test_size=0.2, random_state=42)

# 5. Definir el modelo Keras
input_dim = X_train.shape[1]
num_classes = y_train.shape[1]

model = Sequential([
    Dense(128, activation='relu', input_shape=(input_dim,)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='sigmoid')  # Para clasificación multilabel
])

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

c:\Users\jesus\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
# 6. Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# 7. Predecir etiquetas para nuevos datos
# Libros de prueba
test_books = [
    {
        "text": "Messi: Edición revisada y actualizada. Una biografía del astro argentino.",
        "expected_tags": ["Biography", "Sports", "Football", "Nonfiction", "Sports & Outdoors"]
    },
    {
        "text": "The Great Gatsby. A novel about the American dream and the roaring twenties.",
        "expected_tags": ["Classics", "Fiction", "Literature", "Romance", "Historical Fiction"]
    },
    {
        "text": "A Brief History of Time. Stephen Hawking explains the universe and black holes.",
        "expected_tags": ["Science", "Nonfiction", "Physics", "Philosophy", "History", "Astronomy"]
    },
    {
        "text": "The Catcher in the Rye. A story about teenage rebellion and identity.",
        "expected_tags": ["Classics", "Fiction", "Young Adult", "Literature", "Contemporary Fiction"]
    },
    {
        "text": "The Art of War. Ancient Chinese military strategy by Sun Tzu.",
        "expected_tags": ["Philosophy", "History", "Nonfiction", "Military", "Self-Help"]
    },
    {
        "text": "Harry Potter and the Chamber of Secrets. The second book in the Harry Potter series.",
        "expected_tags": ["Fantasy", "Fiction", "Young Adult", "Adventure", "Magic"]
    }
]

# Predecir etiquetas para los libros de prueba
for book in test_books:
    sample_embedding = model_embed.encode([book["text"]])
    predictions = model.predict(sample_embedding)
    predicted_tags = mlb.inverse_transform((predictions > 0.5).astype(int))[0]
    
    # Comparar etiquetas esperadas y predichas
    expected_tags = set(book["expected_tags"])
    predicted_tags = set(predicted_tags)
    
    # Identificar aciertos y errores
    correct_tags = expected_tags & predicted_tags
    incorrect_tags = predicted_tags - expected_tags
    missing_tags = expected_tags - predicted_tags
    
    # Pintar los resultados
    print(f"Book: {book['text']}")
    print("Expected Tags:", ", ".join(book["expected_tags"]))
    print("Predicted Tags:", ", ".join(
        [colored(tag, "green") if tag in correct_tags else tag for tag in predicted_tags]
    ))
    print("Missing Tags:", ", ".join([colored(tag, "red") for tag in missing_tags]))
    print("-" * 50)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.0000e+00 - loss: 0.6925 - val_accuracy: 0.0000e+00 - val_loss: 0.6880
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.0000e+00 - loss: 0.6872 - val_accuracy: 0.0000e+00 - val_loss: 0.6868
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.0000e+00 - loss: 0.6766 - val_accuracy: 0.5000 - val_loss: 0.6853
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.0000e+00 - loss: 0.6767 - val_accuracy: 0.5000 - val_loss: 0.6836
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.2500 - loss: 0.6687 - val_accuracy: 0.5000 - val_loss: 0.6817
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.0000e+00 - loss: 0.6737 - val_accuracy: 0.5000 - val_loss: 0.6797
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.5000 - loss: 0.6662 - val_accuracy: 0.5000 - val_loss: 0.6778
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.0000e+00 - loss: 0.6577 - val_accura

In [12]:
# 8. Guardar el modelo Keras entrenado y el codificador de etiquetas como "book_tagging_keras_model..."

# Guardar el modelo Keras en el formato recomendado
model.save("../model/book_tagging_keras_model.keras")  # Guardar en formato nativo de Keras

# Guardar el binarizador de etiquetas
joblib.dump(mlb, "../model/book_tagging_keras_mlb_encoder.pkl")  # Guardar el binarizador

['../model/book_tagging_keras_mlb_encoder.pkl']